# Recommendation system for full database (28 mln ratings)

Data loading with chunk_size is 1.0 mln, after that sparse array (lil_sparse) where columns are next movies ando rows - users.

In [1]:
import pandas as pd
import numpy as np
import time
from scipy.sparse import csr_matrix, lil_matrix


# movies_count = 193609+1#193886+1
# users_count = 610+1 #283228+1

users_count = 283228+1
movies_count = 193886+1


cols = ['userId', 'movieId', 'rating']
data_iterator = pd.read_csv("drive/My Drive/ratings.csv", chunksize=10**6, usecols=cols)
chunk_list = []

for data_chunk in data_iterator:
  # if len(chunk_list)<2:
    chunk_list.append(data_chunk)

start = time.time()
ratings = lil_matrix((users_count, movies_count), dtype=np.float32)

for data in chunk_list:
    data_arr = data.to_numpy()
    for user, movie, rating in data_arr:
      # print(user)
      ratings[user, movie] = rating
print(time.time()-start)


689.9590175151825


To calculate cosinus similarity we should normalise our matrix. Because of size, we have to do it col by col. (Even 25GB ram wasnt enough to do it by division [or i did sth incorrectly])


In [0]:
from scipy.sparse import *
from scipy.sparse.linalg import norm
from scipy.sparse import hstack

np.seterr(divide='ignore', invalid='ignore')
r2 = ratings.tocsc()
rl =[]
for i in range(r2.shape[1]):
  rl.append(np.nan_to_num(r2.getcol(i)/norm(r2.getcol(i))))

normalised = hstack(rl)

Let's rate some movies to predict next (here, randomly)

In [4]:
import random
user_rate = csc_matrix((193887, 1))
for _ in range(1000):
  user_rate[random.randint(1, 193000), 0] = random.choice([1, 2, 3, 4, 5])
user_rate_normalised = user_rate / norm(user_rate)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


Counting cosinus similarity array with user ratings array

In [0]:

def recommend_sth(user_rate_normalised):
  Z = normalised * user_rate_normalised
  Z = Z / norm(Z)
  X = normalised
  cos_sim = X.T * Z

  res_en = [(el[0], i) for i, el in enumerate(cos_sim.toarray())]
  res_en.sort(key=lambda x: x[0])
  res_en = res_en[::-1]
  return res_en

# res = recommand_sth(user_rate_normalised)

In [0]:
movies_data = pd.read_csv('drive/My Drive/movies.csv', usecols=['movieId', 'title']).to_numpy()
movies = np.array([None for _ in range(movies_count+1)])

for id, title in movies_data:
  movies[id] = title

Final recommendation list, sorted

In [14]:
result_rank = recommend_sth(user_rate_normalised)
movies_recommandation = list(map(lambda x: movies[x[1]], result_rank))
movies_recommandation

['People That Are Not Me (2016)',
 'Identification Marks: None (Rysopis) (1965) ',
 'Beauty #2 (1965)',
 'Age of the Earth, The (A Idade da Terra) (1980)',
 'Remote Control (1992)',
 'All the Wrong Reasons (2013)',
 'Ma (2015)',
 'How He Fell in Love (2015)',
 'Vazante (2017)',
 'Mad (2016)',
 'Ernesto (2017)',
 'There Is No Place Like Home (2018)',
 'Chrystal (2004)',
 'Cops vs. Thugs (1975)',
 'Still Alive (2016)',
 'Boulevard Nights (1979)',
 "I'm the King of the Castle (1989)",
 'The Skirt Chaser (1979)',
 'Le Complot (1973)',
 'Black Cop (2017)',
 'Galaxy Turnpike (2015)',
 'Nobunaga Concerto: The Movie (2016)',
 'Grasshopper (2015)',
 'Unpaid Vacation (1982)',
 'Under the Tree (2017)',
 'Celtic Soul (2016)',
 'Love Rites (1987)',
 'Napalm (2017)',
 'Dim the Fluorescents (2017)',
 'The Night of the Virgin (2016)',
 'In Times of Fading Light (2017)',
 'Memoir of Pain (2018)',
 "Nobody's Watching (2017)",
 'Mektoub, My Love: Canto Uno (2018)',
 'Nine Meals from Chaos (2018)',
 'Euge